### Building a Chain of thought of RAG Pipeline using Ollama x llama2, Qdrant and DSPy.

#### Author: Sachin Khandewal

## Setup

In [ ]:
# !pip install dspy-ai
# !pip install qdrant-client
# !pip install ollama

## Load the dataset

In [1]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=1000)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
dataset = [x.with_inputs('question') for x in dataset.train]

len(dataset)

c:\Users\ASUS\anaconda3\envs\new\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


1000

In [5]:
dataset[9]

Example({'question': 'What is the code name for the German offensive that started this Second World War engagement on the Eastern Front (a few hundred kilometers from Moscow) between Soviet and German forces, which included 102nd Infantry Division?', 'answer': 'Operation Citadel'}) (input_keys={'question'})

In [6]:
dataset[300].question, dataset[300].answer

('What is a block away from the arena where the Baltimore Blast play their games?',
 'Baltimore Convention Center')

In [7]:
# import torch
# torch.cuda.is_available()

## Initialize the retriever using Qdrant

In [8]:
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient
 
qdrant_client = QdrantClient(":memory:")  # loads in memory for this session

docs = [x.question + " -> " + x.answer for x in dataset]

ids = list(range(0,len(docs)))


qdrant_client.add(
    collection_name="hotpotqa",
    documents=docs,
    ids=ids
    )


qdrant_retriever_model = QdrantRM("hotpotqa", qdrant_client, k=3)

2024-03-20 20:03:08.472 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated. Use TextEmbedding instead.
100%|██████████| 77.7M/77.7M [00:23<00:00, 3.35MiB/s]


## Load the Ollama model

Pre-requisites: should have downlaoded the Ollama application from: https://ollama.com/download/windows

For Windows you can follow these steps:
1) To download it, go to: Download Ollama on Windows. 
2) Install it on your system.
3) After installing, you can open the command prompt and type “ollama pull llama2”, which will download the latest quantized image for Llama2; by default, it pulls a 7B model.
4) You will see the Ollama icon in your hidden icons, which means that you can run Ollama models now!


In [9]:
import dspy
ollama_model = dspy.OllamaLocal(model="llama2",model_type='text',
                                max_tokens=350,
                                temperature=0.7,
                                top_p=0.9, frequency_penalty=1.17, top_k=40)

Try out one example before proceeding

In [10]:
ollama_model("tell me about interstellar's plot")

["Interstellar is a 2014 science fiction film directed by Christopher Nolan that follows a group of astronauts on a mission to travel through wormholes in search of a new habitable planet for humanity. Here is the general plot outline:\n\nIn the near future, Earth is facing an unprecedented crisis as crops are failing and food supplies are running low. The world's top scientists, led by Dr. Josep Mendes (Michael Caine), are unable to find a solution to this problem, which threatens the survival of humanity.\n\nEnter Cooper (Matthew McConaughey), a former NASA pilot who is recruited for a dangerous mission through wormholes in search of a new habitable planet. Cooper's daughter Murph (Jessica Chastain) is a brilliant young girl who has been struggling to understand the equations that could potentially save Earth, and her father's journey becomes crucial to her research.\n\nCooper and his team, which includes Dr. Stone (Stephen Henderson), Amelia Brand (Anne Hathaway), and Romilly (David

## Get Top Passages from the retriever

In [11]:
import dspy

dspy.settings.configure(rm=qdrant_retriever_model, lm=ollama_model)

dev_example = dataset[100]

def get_top_passages(question):
    retrieve = dspy.Retrieve(k=3)
    topK_passages = retrieve(question,k=3).passages
    print(f"Top {retrieve.k} passages for question: {question} \n", '-' * 30, '\n')
    for idx, passage in enumerate(topK_passages):
        print(f'{idx+1}]', passage, '\n')

get_top_passages(dev_example.question)

Top 3 passages for question: What was a previous unoffical name for the high performance variant of Audi's compact executive car? 
 ------------------------------ 

1] What was a previous unoffical name for the high performance variant of Audi's compact executive car? -> Audi Ur-S4 

2] What car models used the same Saxomat clutch as the automobiles produced by former East German auto maker VEB Sachsenring Automobilwerke Zwickau in Zwickau, Saxony? -> Fiat 1800, Lancia Flaminia, Saab 93, Borgward Isabella, Goliath/Hansa 1100, Auto Union 1000, Ford Taunus 

3] William Sachiti is the founder of the company that is a UK competitor to the major automaker based in what city? -> Palo Alto 



## Define Signatures for Input and Output

In [12]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may or maynot contain relevant facts or answer keywords")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="an answer between 10 to 20 words")

In [13]:
ga = GenerateAnswer(context="My name is sachin and I like writing blogs", question="What is my name?", answer="Sachin")

In [14]:
print(ga.model_construct)

<bound method BaseModel.model_construct of GenerateAnswer(context, question -> answer
    instructions='Answer questions with short factoid answers.'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may or maynot contain relevant facts or answer keywords', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'an answer between 10 to 20 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)>


## Create a DSPy CoT Module

In [15]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

Let's call the RAG object

In [130]:
uncompiled_rag = RAG()

Define some datapoints from which we can ask CoT questions

In [19]:
dataset[91],dataset[100],dataset[6]

(Example({'question': "Who has been on a British television music competition show and was was most popular in the 80's with the pop band 'Culture Club'?", 'answer': "George Alan O'Dowd"}) (input_keys={'question'}),
 Example({'question': "What was a previous unoffical name for the high performance variant of Audi's compact executive car?", 'answer': 'Audi Ur-S4'}) (input_keys={'question'}),
 Example({'question': 'Which is taller, the Empire State Building or the Bank of America Tower?', 'answer': 'The Empire State Building'}) (input_keys={'question'}))

## Let's run the queries now

In [133]:
my_question = "Was George Alan O'Dowd the most popular in the late 2000s with his rock band?"
# my_question = "which segment of Audi's car was named as Ur-S4?"
# my_question = "is Bank of America Tower taller than empire state building?"

# get_top_passages(my_question)

response = uncompiled_rag(my_question)
print(response.answer)

No, George Alan O'Dowd was not the most popular in the late 2000s with his rock band.


To inspect the prompt internally

In [134]:
ollama_model.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Context: may or maynot contain relevant facts or answer keywords

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: an answer between 10 to 20 words

---

Context:
[1] «Who has been on a British television music competition show and was was most popular in the 80's with the pop band 'Culture Club'? -> George Alan O'Dowd»
[2] «Who was dubbed the father of the type of rock music that emerged from post-punk in the late 1970s? -> Brian Healy»
[3] «Alan Forbes has done posters for an American rock band that formed in 1996 in what city in California? -> Palm Desert»

Question: Was George Alan O'Dowd the most popular in the late 2000s with his rock band?

Reasoning: Let's think step by step in order to Answer: No, George Alan O'Dowd was not the most popular in the late 2000s. He was active and popular in the 1980s with his pop band Culture Club.

Ans

## ADVANCED (Using teleprompters to train the CoT model)

In [98]:
# compile

dataset_dev = [dataset[91],
dataset[100],
dataset[6]]

from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), dataset=dataset_dev)

100%|██████████| 3/3 [02:44<00:00, 54.95s/it]

Bootstrapped 1 full traces after 3 examples in round 0.


In [102]:
# Ask any question you like to this simple RAG program.

my_question = "Was George Alan O'Dowd the most popular in the early 1980s with his pop band?"
# my_question = "which segment of Audi's car was named as Ur-S4?"
# my_question = "is Bank of America Tower taller than empire state building?"

# get_top_passages(my_question)

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

In [101]:
ollama_model.inspect_history(1)





Given the fields `context`, `question`, produce the fields `answer`.

---

Question: Which is taller, the Empire State Building or the Bank of America Tower?
Answer: The Empire State Building

Question: What was a previous unoffical name for the high performance variant of Audi's compact executive car?
Answer: Audi Ur-S4

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context:
[1] «Who has been on a British television music competition show and was was most popular in the 80's with the pop band 'Culture Club'? -> George Alan O'Dowd»
[2] «The Fridge had booked such acts as the English singer and DJ who was the lead singer of what pop band? -> Culture Club»
[3] «Celebration was recorded by the singer who was a leading presence when what cable channel premiered? -> MTV»

Question: Who has been on a British television music competition show and was was 